In [17]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [18]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import peakutils
import locate
import farwave
import time

from scipy import signal

MIC_IND_LR = {
    0: (1, 2),
    1: (2, 0),
    2: (0, 1)
}

ANGLE_OFFSET = {
    0: 0,
    1: 2.0944, # 120 deg in rad
    2: 4.18879, # 240 deg in rad
}


In [19]:
%matplotlib notebook

In [20]:
%sql sqlite:///schema/test_low.db

u'Connected: None@schema/test_low.db'

In [21]:
l = 0.3 # Length from mic to centre of array

def calc_delays(sigs, offsets):
    """ Calculate matrix of delays given N sigs
    """
    results = []
    for j in range(len(sigs)):
        for k in range(len(sigs)):
            if j != k:
                results.append((
                        (j, k), locate.xcorr_peaks(sigs[j], sigs[k], offsets[j], offsets[k], l)
                    ))
            

    # 3x3 array delays[i][j] is the delay of signal j relative to signal i
    delays = np.zeros((len(sigs), len(sigs)), dtype=np.float)
    for res in results:
        key, val = res[0], res[1]
        delays[key[0]][key[1]] = val[1] # xcorr (val, delay) tuple
        delays[key[1]][key[0]] = -delays[key[0]][key[1]]
        
    return delays

def calc_angles(sigs, delays, l):
    """ Given delays matrix, calculate angle using both farwave and solver
    """
    # Estimate "location" of sound source, create array record
    theta = 0             
    for j in range(len(sigs)):
        if delays[j][(j+1)%3] >= 0 and delays[j][(j+2)%3] >= 0:
            lr = MIC_IND_LR[j]
            near_pair = (min(j, lr[0]), max(j, lr[0])) if delays[j][lr[0]] < delays[j][lr[1]] else (min(j, lr[1]), max(j, lr[1]))
            print 'Near Pair: %r' %  str(near_pair)
            f_theta = np.deg2rad(farwave.calc_angle(delays, l, near_pair=near_pair))
            print 'Using microphone %d as closest mic - (%d left, %d right)' % (j, lr[0], lr[1])
            r, theta = 0, 0 #locate.locate(delays[j][lr[0]], delays[j][lr[1]], l, r0=5., theta0=f_theta-ANGLE_OFFSET[j])
            theta += ANGLE_OFFSET[j]
            print 'Solver: r = %f, theta = %f' % (r, np.rad2deg(theta))
            break
    
    return np.rad2deg(theta), np.rad2deg(f_theta)


In [22]:
experiments = %sql select id, comment from experiment;

Done.


In [23]:
ang_solver_raw = []
ang_solver_butter = []
ang_far_raw = []
ang_far_butter = []

for i in range(len(experiments)):
    cur_experiment_id = int(experiments[i][0])
    result = %sql select data from mic where experiment_id = :cur_experiment_id and array_id = 0
        
    mic_0 = np.array(result[0][0])
    mic_1 = np.array(result[1][0])
    mic_2 = np.array(result[2][0])
    
    sigs = [mic_0, mic_1, mic_2]
    #try:
    try: 
        sigs_cropped, sigs_butter_cropped, offsets, _, _ = locate.crop_sigs_npeaks(sigs)
    except RuntimeError as e:
        print 'Error: %s' % str(e)
        solver_raw = solver_butter = far_raw = far_butter = 0
        ang_solver_raw.append(solver_raw)
        ang_solver_butter.append(solver_butter)
        ang_far_raw.append(far_raw)
        ang_far_butter.append(far_butter)
        continue
        
    # Calculate delays, angle using raw signal (cropped)
    delays_raw = calc_delays(sigs_cropped, offsets)
    solver_raw, far_raw = calc_angles(sigs_cropped, delays_raw, l)

    delays_butter = calc_delays(sigs_butter_cropped, offsets)
    solver_butter, far_butter = calc_angles(sigs_butter_cropped, delays_butter, l)
    #except:
    #    print 'Failure to find peaks: Setting to 0'
    #    solver_raw = far_raw = solver_butter = far_butter = 0
        
    print 'ID: %d, %f,%f,%f,%f' % (cur_experiment_id, far_raw, far_butter, solver_raw, solver_butter)
    
    ang_solver_raw.append(solver_raw)
    ang_solver_butter.append(solver_butter)
    ang_far_raw.append(far_raw)
    ang_far_butter.append(far_butter)


Done.
Near Pair: '(0, 1)'
Angles: [54.88267050621118, 65.5262735378872, 46.54321711800405]
Errors: [1.0040018175373786, 1.72094464334052, 1.4538957084015323]

DEL_ERR: 0.716943
Error: 0.716943, 1.004002, 'True'
DEL_ERR: 0.449894
Error: 0.449894, 1.004002, 'True'
DEL_ERR: 0.267049
Error: 0.267049, 1.004002, 'False'
Angle (CCW): 54.886701
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.000000, theta = 0.000000
Near Pair: '(0, 1)'
Angles: [54.25236598240461, 66.2065701661745, 45.00422063838648]
Errors: [1.0050527473193134, 1.692911883236332, 1.4143177505645514]

DEL_ERR: 0.687859
Error: 0.687859, 1.005053, 'True'
DEL_ERR: 0.409265
Error: 0.409265, 1.005053, 'True'
DEL_ERR: 0.278594
Error: 0.278594, 1.005053, 'False'
Angle (CCW): 54.255426
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.000000, theta = 0.000000
ID: 98, 54.886701,54.255426,0.000000,0.000000
Done.
Near Pair: '(0, 1)'
Angles: [55.58360402253258, 69.96892267830569, 43.61988023701859]
Err

In [24]:
# Assuming comment is the _actual_ angle
for i in range(len(ang_solver_raw)):
    print '%f,%f,%f,%f,%s' % (ang_far_raw[i], ang_far_butter[i], ang_solver_raw[i], ang_solver_butter[i], experiments[i][1])

54.886701,54.255426,0.000000,0.000000,-55
55.654999,54.567230,0.000000,0.000000,-55
56.797383,55.191413,0.000000,0.000000,-55
38.360418,36.324752,0.000000,0.000000,-25
31.395347,28.077765,0.000000,0.000000,-25
39.902011,37.930973,0.000000,0.000000,-25
0.402545,0.792866,0.000000,0.000000,5
359.051293,0.269493,0.000000,0.000000,5
1.804829,2.413858,0.000000,0.000000,5
330.611005,331.439944,0.000000,0.000000,40
330.900315,331.236646,0.000000,0.000000,40
329.177086,330.523922,0.000000,0.000000,40
298.363011,299.561182,239.999988,239.999988,70
296.905086,296.849797,239.999988,239.999988,70
298.070635,298.553234,239.999988,239.999988,70
